In [1]:
from twosigmafunc import R_score, split_data, preprocess, add_diffs, add_nans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
plt.rcParams['figure.figsize'] = (14.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
train, origin_features = preprocess()

In [4]:
add_nans(train)

sucessfully add 108 nan features


In [5]:
add_diffs(train, origin_features)

sucessfully add 108 diff features


In [8]:
train.fillna(train.median(), inplace=True)
train.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_35_diff,technical_36_diff,technical_37_diff,technical_38_diff,technical_39_diff,technical_40_diff,technical_41_diff,technical_42_diff,technical_43_diff,technical_44_diff
131062,0,167,-0.000837,0.005523,0.021095,0.002476,0.011752,-0.040645,-0.007395,-0.030291,...,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0
131895,0,168,-0.000837,0.005523,0.021095,0.002476,0.011752,-0.040645,-0.007395,-0.030291,...,0.0,0.0,4.400551e-18,1.388941e-19,8.286639e-23,0.0,0.0,0.0,0.0,0.0
132728,0,169,-0.000837,0.005523,0.021095,0.002476,0.011752,-0.040645,-0.007395,-0.030291,...,0.0,0.0,4.400551e-18,1.388941e-19,8.286639e-23,0.0,0.0,0.0,0.0,0.0
133561,0,170,-0.230583,0.488096,0.935920,0.028222,-0.083071,-0.240929,-0.007395,0.212425,...,0.0,0.0,4.400551e-18,1.388941e-19,8.286639e-23,0.0,0.0,0.0,0.0,0.0
134393,0,171,-0.230583,0.488096,0.935920,0.028222,-0.083071,-0.240929,-0.007395,0.212425,...,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0


# Add some important features 

In [10]:
train['id_diff'] = train.id.diff()
train['tec20-30'] = train.technical_20 - train.technical_30
train['tec123'] = train['tec20-30'] + train.technical_13
train['y_past'] = train.y.shift()
train['tec123_past'] = train.tec123.shift()
train.loc[train.id_diff != 0, ['y_past', 'tec123_past']] = 0
train.fillna(0, inplace=True)

In [11]:
lr_features = ['tec20-30', 'technical_20_diff']

In [15]:
from sklearn.linear_model import Ridge
import twosigmafunc

In [13]:
ridge_1 = Ridge(normalize=True)
ridge_2 = Ridge(normalize=True)

In [16]:
X_train, y_train, X_test, y_test = split_data(train, twosigmafunc.origin_features(train, ['id', 'timestamp', 'y', 'id_diff']))

In [17]:
low_y_cut = -0.075
high_y_cut = 0.075
y_is_above_cut = (y_train > high_y_cut)
y_is_below_cut = (y_train < low_y_cut)
y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)

In [18]:
X_train = X_train[y_is_within_cut]
y_train = y_train[y_is_within_cut]
num = y_train.shape[0]

In [19]:
num

791870

In [36]:
better = True
r_best_train = -99999
r_best_test = -99999
quant = 0.99
limit_len = num * 0.9
ix = X_train.index
while better:
    print('training on', len(ix), 'samples')
    ridge_2.fit(X_train.loc[ix, lr_features], y_train.loc[ix])
    y_pred_residual = ridge_1.predict(X_train.loc[ix, lr_features])
    y_train_pred = ridge_2.predict(X_train[lr_features])
    y_test_pred = ridge_2.predict(X_test[lr_features])
    r = R_score(y_test_pred, y_test)
    r_train = R_score(y_train_pred, y_train)
    print('r_score on train:', r_train, ', improving:', r_train - r_best_train)
    print('r_score on test:', r, ', improving:', r - r_best_test)
    if r > r_best_test:
        r_best_test = r 
    if r_train > r_best_train:
        r_best_train = r_train
        residual = np.abs(y_pred_residual - y_train[ix])
        ix = residual[abs(residual) <= abs(residual).quantile(quant)].index
        if len(ix) < limit_len:
            break
    else:
        break

training on 791870 samples
r_score on train: 0.0223489496849 , improving: 99999.0223489
r_score on test: 0.0246343621827 , improving: 99999.0246344
training on 783951 samples
r_score on train: 0.0231251924926 , improving: 0.000776242807643
r_score on test: 0.025717726297 , improving: 0.00108336411427
training on 776111 samples
r_score on train: 0.0231663033699 , improving: 4.11108773756e-05
r_score on test: 0.0258679325651 , improving: 0.000150206268154
training on 768349 samples
r_score on train: 0.0230078719005 , improving: -0.000158431469423
r_score on test: 0.0258765475687 , improving: 8.61500361838e-06


In [39]:
better = True
r_best_train = -99999
r_best_test = -99999
quant = 0.99
limit_len = num * 0.9
ix = X_train.index
while better:
    print('training on', len(ix), 'samples')
    ridge_1.fit(np.array(X_train.loc[ix, lr_features[1]]).reshape(-1, 1), y_train.loc[ix])
    y_pred_residual = ridge_1.predict(np.array(X_train.loc[ix, lr_features[1]]).reshape(-1, 1))
    y_train_pred = ridge_1.predict(np.array(X_train[lr_features[1]]).reshape(-1,1))
    y_test_pred = ridge_1.predict(np.array(X_test[lr_features[1]]).reshape(-1,1))
    r = R_score(y_test_pred, y_test)
    r_train = R_score(y_train_pred, y_train)
    print('r_score on train:', r_train, ', improving:', r_train - r_best_train)
    print('r_score on test:', r, ', improving:', r - r_best_test)
    if r > r_best_test:
        r_best_test = r 
    if r_train > r_best_train:
        r_best_train = r_train
        residual = np.abs(y_pred_residual - y_train[ix])
        ix = residual[abs(residual) <= abs(residual).quantile(quant)].index
        if len(ix) < limit_len:
            break
    else:
        break

training on 791870 samples
r_score on train: 0.0149244686904 , improving: 99999.0149245
r_score on test: 0.0198638321117 , improving: 99999.0198638
training on 783951 samples
r_score on train: 0.0155413209696 , improving: 0.000616852279273
r_score on test: 0.0208503996251 , improving: 0.000986567513408
training on 776111 samples
r_score on train: 0.0155900364915 , improving: 4.87155218904e-05
r_score on test: 0.0209409604868 , improving: 9.05608617213e-05
training on 768349 samples
r_score on train: 0.0156443889891 , improving: 5.4352497612e-05
r_score on test: 0.0211267717193 , improving: 0.000185811232517
training on 760665 samples
r_score on train: 0.0155475936357 , improving: -9.67953534738e-05
r_score on test: 0.0210691261756 , improving: -5.76455437521e-05


# ETR features 

In [46]:
import json
from sklearn.ensemble import ExtraTreesRegressor

In [125]:
try:
    with open('features.json', 'r') as file:
        features = json.load(file)
except IOError as err:
    raise err

In [49]:
etr = ExtraTreesRegressor(n_jobs=-1,n_estimators=128, max_depth=6, 
                          max_features=0.6, min_samples_leaf=30,
                          random_state=17, verbose=0)
etr.fit(X_train[etr_features], y_train)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=6,
          max_features=0.6, max_leaf_nodes=None, min_impurity_split=1e-07,
          min_samples_leaf=30, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=128, n_jobs=-1,
          oob_score=False, random_state=17, verbose=0, warm_start=False)

In [50]:
ridge_1.fit(np.array(X_train[lr_features[1]]).reshape(-1, 1), y_train)    

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [51]:
ridge_2.fit(X_train[lr_features], y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [52]:
del train

In [53]:
X_train['y_lr1'] = ridge_1.predict(np.array(X_train[lr_features[1]]).reshape(-1, 1))
X_train['y_lr2'] = ridge_2.predict(X_train[lr_features])
X_test['y_lr1'] = ridge_1.predict(np.array(X_test[lr_features[1]]).reshape(-1, 1))
X_test['y_lr2'] = ridge_2.predict(X_test[lr_features])

In [54]:
X_train['y_etr'] = etr.predict(X_train[etr_features])
X_test['y_etr'] = etr.predict(X_test[etr_features])

In [57]:
residual = pd.DataFrame({
        'residual_lr1':np.abs(X_train['y_lr1'] - y_train),
        'residual_lr2':np.abs(X_train['y_lr2'] - y_train),
        'residual_etr':np.abs(X_train['y_etr'] - y_train),
    })

In [60]:
residual['best'] = residual.idxmin(axis=1)

In [62]:
residual.head()

,residual_etr,residual_lr1,residual_lr2,best
131062,0.006954,0.007190,0.007202,residual_etr
131895,0.002038,0.001868,0.001856,residual_lr2
132728,0.017888,0.017643,0.017631,residual_lr2
133561,0.012839,0.012853,0.012841,residual_etr
134393,0.025352,0.025311,0.025323,residual_lr1


In [64]:
model_features = etr_features + ['y_lr1', 'y_lr2', 'y_etr']

In [65]:
from sklearn.ensemble import ExtraTreesClassifier
model_etr = ExtraTreesClassifier(n_jobs=-1,n_estimators=128, max_depth=6, 
                          max_features=0.6,
                          random_state=17, verbose=0)

In [66]:
model_etr.fit(X_train[model_features], residual.best)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=6, max_features=0.6, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=128, n_jobs=-1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [67]:
best_model_train = model_etr.predict(X_train[model_features])

In [69]:
np.mean(best_model_train == residual.best)

0.47803553613598193

In [70]:
residual_test = pd.DataFrame({
        'residual_lr1':np.abs(X_test['y_lr1'] - y_test),
        'residual_lr2':np.abs(X_test['y_lr2'] - y_test),
        'residual_etr':np.abs(X_test['y_etr'] - y_test),
    })

In [71]:
residual_test['best'] = residual_test.idxmin(axis=1)

In [73]:
best_model_test = model_etr.predict(X_test[model_features])
probs = model_etr.predict_proba(X_test[model_features])

In [77]:
y_basic = 0.75 * X_test.y_etr + 0.2 * X_test.y_lr2 + 0.05 * X_test.y_lr1

In [75]:
probs

array([[ 0.47331326,  0.1480471 ,  0.37863964],
       [ 0.48265545,  0.21398561,  0.30335894],
       [ 0.4930417 ,  0.30018219,  0.20677612],
       ..., 
       [ 0.44041989,  0.24420038,  0.31537973],
       [ 0.45458804,  0.24376482,  0.30164714],
       [ 0.47191509,  0.32809474,  0.19999016]])

In [76]:
R_score(y_basic, y_test)

0.025406067236916314

In [80]:
y_model = probs[:,0] * X_test.y_etr + probs[:,2] * X_test.y_lr2 + probs[:,1] * X_test.y_lr1

In [81]:
R_score(y_model, y_test)

0.025596126615695682

In [85]:
imp = pd.DataFrame({'feature':model_features, 'important':model_etr.feature_importances_}).sort_values('important', ascending=False)

,feature,important
10,technical_20,0.245408
1,tec20-30,0.119768
3,tec123,0.068962
32,technical_20_diff,0.063568
98,y_lr1,0.062851
99,y_lr2,0.062715
41,technical_22,0.057385
6,tec123_past,0.053950
12,fundamental_25_nan,0.049425
23,technical_7,0.046965


In [92]:
model_features = imp.loc[imp.important > imp.important.mean() / 10, 'feature']

In [88]:
len(model_features)

26

In [93]:
model_etr.fit(X_train[model_features], residual.best)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=6, max_features=0.6, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=128, n_jobs=-1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [94]:
probs = model_etr.predict_proba(X_test[model_features])
y_model = probs[:,0] * X_test.y_etr + probs[:,2] * X_test.y_lr2 + probs[:,1] * X_test.y_lr1
R_score(y_model, y_test)

0.025590265591563088

In [95]:
y_model = (probs[:,0] + 0.75) / 2 * X_test.y_etr + (probs[:,2] + 0.2) / 2 * X_test.y_lr2 + (probs[:,1] + 0.05) / 2 * X_test.y_lr1
R_score(y_model, y_test)

0.025728764241391578

In [97]:
y_model = (probs[:,0] * 2 + 0.75) / 3 * X_test.y_etr + (probs[:,2] * 2 + 0.2) / 3 * X_test.y_lr2 + (probs[:,1] * 2 + 0.05) / 3 * X_test.y_lr1
R_score(y_model, y_test)

0.025733699498178265

set `max_features = 0.8`

In [98]:
model_etr = ExtraTreesClassifier(n_jobs=-1,n_estimators=128, max_depth=6, max_features=0.8,
                          random_state=17, verbose=0)
model_etr.fit(X_train[model_features], residual.best)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=6, max_features=0.8, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=128, n_jobs=-1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [100]:
probs = model_etr.predict_proba(X_test[model_features])
y_model = (probs[:,0] + 0.75) / 2 * X_test.y_etr + (probs[:,2] + 0.2) / 2 * X_test.y_lr2 + (probs[:,1] + 0.05) / 2 * X_test.y_lr1
R_score(y_model, y_test)

0.025740364077357233

In [117]:
model_etr = ExtraTreesClassifier(n_jobs=-1,n_estimators=128, max_depth=5, max_features=0.8,
                          random_state=17, verbose=0)
model_etr.fit(X_train[model_features], residual.best)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=5, max_features=0.8, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=128, n_jobs=-1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [118]:
best_model_train = model_etr.predict(X_train[model_features])
np.mean(best_model_train == residual.best)

0.47610340081073915

In [119]:
probs = model_etr.predict_proba(X_test[model_features])
y_model = (probs[:,0] + 0.75) / 2 * X_test.y_etr + (probs[:,2] + 0.2) / 2 * X_test.y_lr2 + (probs[:,1] + 0.05) / 2 * X_test.y_lr1
R_score(y_model, y_test)

0.025709521624548957

In [120]:
model_etr = ExtraTreesClassifier(n_jobs=-1,n_estimators=128, max_depth=5, max_features=0.8,
                          random_state=17, verbose=0, min_samples_leaf=10)
model_etr.fit(X_train[model_features], residual.best)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=5, max_features=0.8, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=10,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=128, n_jobs=-1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [121]:
probs = model_etr.predict_proba(X_test[model_features])
y_model = (probs[:,0] + 0.75) / 2 * X_test.y_etr + (probs[:,2] + 0.2) / 2 * X_test.y_lr2 + (probs[:,1] + 0.05) / 2 * X_test.y_lr1
R_score(y_model, y_test)

0.025707923593783009

In [123]:
list(model_features)

['technical_20',
 'tec20-30',
 'tec123',
 'technical_20_diff',
 'y_lr1',
 'y_lr2',
 'technical_22',
 'tec123_past',
 'fundamental_25_nan',
 'technical_7',
 'technical_40',
 'y_etr',
 'technical_30',
 'fundamental_27_nan',
 'y_past',
 'technical_30_diff',
 'fundamental_17_nan',
 'technical_2',
 'fundamental_63_nan',
 'fundamental_59',
 'fundamental_47_nan',
 'fundamental_5_nan',
 'technical_17',
 'technical_9_nan',
 'technical_11',
 'technical_21']

# Training a Network to combine 3 model results? 

In [126]:
nn_features = features['nn_features']

In [128]:
nn_features.extend(['y_lr1', 'y_lr2', 'y_etr'])

In [130]:
len(nn_features)

35

In [131]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using Theano backend.


In [141]:
# define base mode
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(35, input_dim=35, init='normal', activation='relu'))
	model.add(Dense(1, init='normal', activation='relu'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
baseline_model().fit(np.array(X_train[nn_features]), np.array(y_train), nb_epoch=10)

Epoch 1/10
791870/791870 [==============================] - 36s - loss: 3.4753e-04 - acc: 0.0000e+00    
Epoch 2/10
791870/791870 [==============================] - 31s - loss: 3.4731e-04 - acc: 0.0000e+00    
Epoch 3/10
141440/791870 [====>.........................] - ETA: 24s - loss: 3.4600e-04 - acc: 0.0000e+00